In [22]:
import pandas as pd
import numpy as np
import re

%run preprocessor.ipynb

In [41]:
def get_keywords(df, query):
    # Create the regex pattern for the keywords
    query_pattern = r'\b(?:' + '|'.join(query) + r')\b'
    
    # Function to extract excerpts
    def extract_excerpts(script, pattern):
        excerpts = []
        for match in re.finditer(pattern, script, re.IGNORECASE):
            start = max(match.start() - 100, 0)
            end = match.end() + 100
            excerpt = script[start:end]
            excerpts.append(excerpt)
        return excerpts
    
    # Apply the function to each row and create a new column 'excerpts'
    df['excerpts'] = df['script'].apply(lambda x: extract_excerpts(x, query_pattern))
    
    # Filter rows where excerpts list is not empty
    df = df[df['excerpts'].str.len() > 0]
    
    # Add the 'query' column to indicate rows where keywords were found
    df['query'] = True
    
    return df

In [23]:
er_df = pd.read_csv('./data/transcripts/er_transcripts.csv', index_col=0)

er_df = clean_transcripts(er_df)

In [51]:
abortion_keys = ['abortion', 'abort', 'abortifacient']

er_abortion = get_keywords(er_df, abortion_keys)

er_abortion = er_abortion.drop([83, 111, 215])

/var/folders/8b/76lk071d1dn3ck38wkf1wknw0000gn/T/ipykernel_22013/2477543652.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['query'] = True
